In [7]:
import sqlite3
import sql_queries as sq
import pandas as pd

db_conn = sqlite3.connect('./../../_data_raw/fa_database.db')

In [16]:
# create tables in create_table_list
for table in sq.create_table_list:
    cursor = db_conn.cursor()
    
    try:
        cursor.execute(table)
        db_conn.commit()
        cursor.close()
    except Exception as e:
        print(e)
        db_conn.rollback()
        cursor.close()

In [2]:
# drop tables in create_table_list
#for table in sq.drop_table_list:
#    cursor = db_conn.cursor()
#    
#    try:
#        cursor.execute(table)
#        db_conn.commit()
#        cursor.close()
#    except Exception as e:
#        print(e)
#        db_conn.rollback()
#        cursor.close()

# DROP TABLE MANUALLY
# table_to_drop = 'ov_revenue'
# cursor = db_conn.cursor()
# cursor.execute('drop table if exists %s' %table_to_drop)
# cursor.commit()

In [10]:
## test extract and load data from bigquery to local db
from  class_ETL import ETL

worker = ETL(
            credentials_file_path= './../../_config/cred.json',
             local_database= './../../_data_raw/fa_database.db'
             )


In [2]:
df_bigquery_data = worker.extract_bigquery_data(start_date= '2024-01-01')
df_bigquery_data

In [11]:
df_fct_fa_pnl_mini = worker.extract_local_data(table_name='fct_fa_pnl_mini')
df_fct_fa_pnl_mini

In [4]:
# test load data to local db with specific start_date to insert
worker.load_data(data_ingest= df_bigquery_data.to_records(index=False).tolist(), table_name= 'fct_fa_pnl_mini' )

In [3]:
# test load data automatically from lastest complete_date in fct_fa_pnl_mini to current_date - 1 day
worker.auto_ingest_data(table_name='fct_fa_pnl_mini')

In [10]:
import requests

res = requests.get(f'https://api.vietqr.io/v2/business/0107370860',timeout=1000)
res
res.json()